# SDXL Dreambooth training with Kohya Scripts - No config version

In [ ]:
# 1. INSTALL DEPENDENCIES

print("Installing dependencies - this might take a few minutes...")
%cd sd-scripts
!apt-get update -y && apt-get install -y libgl1
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118 # no_verify leave this to specify not checking this a verification stage
!pip install xformers==0.0.20 bitsandbytes==0.35.0
!pip install tensorboard==2.12.3 tensorflow==2.12.0 wheel
!pip install protobuf==3.20.3
!pip install tensorrt
# !pip install --upgrade -r requirements.txt
!pip install gdown
!pip install accelerate==0.19.0
!pip install transformers==4.30.2
!pip install diffusers[torch]==0.18.2
!pip install ftfy==6.1.1
!pip install albumentations==1.3.0
!pip install opencv-python==4.7.0.68
!pip install einops==0.6.0
!pip install pytorch-lightning==1.9.0
!pip install bitsandbytes==0.35.0
!pip install tensorboard==2.12.0
!pip install safetensors==0.3.1
!pip install # gradio==3.16.2
!pip install altair==4.2.2
!pip install easygui==0.98.3
!pip install toml==0.10.2
!pip install voluptuous==0.13.1
!pip install huggingface-hub==0.15.1
!pip install invisible-watermark==0.2.0 
!pip install open-clip-torch==2.20.0
!pip install protobuf==3.20.3
!pip install -e .
print("Finished installing dependencies.")

In [2]:
token_word = "ohwx"
class_word = "person"
training_repeats = 40

In [6]:
# creating parent folders
import os

training_dir = f'training_images/{training_repeats}_{token_word} {class_word}'
reg_dir = f'reg_images/1_{class_word}'

if os.path.exists(training_dir) == False:
  os.makedirs(training_dir)
  print(f'{training_dir} Created.')
else:
  print(f'{training_dir} already exists.')

if os.path.exists(reg_dir) == False:
  os.makedirs(reg_dir)
  print(f'{reg_dir} Created.')
else:
  print(f'{reg_dir} already exists.')





training_images/40_ohwx person already exists.
reg_images/1_person Created.


In [11]:
# download training images
!gdown 1BIixbqMYW5-eOvTFC_RaZyWmlnBzey-0 -O train.zip

# move regs.zip to reg_images folder
shutil.move('train.zip', f'{training_dir}')

# extract zip contents to current folder and delete zip
import zipfile
%cd $training_dir
with zipfile.ZipFile('train.zip', 'r') as train_ref:
    train_ref.extractall()

# delete zip
!rm train.zip


Downloading...
From: https://drive.google.com/uc?id=1BIixbqMYW5-eOvTFC_RaZyWmlnBzey-0
To: /workspace/sd-scripts/train.zip
100%|██████████████████████████████████████| 16.2M/16.2M [00:00<00:00, 70.8MB/s]
/workspace/sd-scripts/training_images/40_ohwx person


In [12]:
%cd /workspace/sd-scripts

# download regs images
!gdown 1CIqOhLBfzrl5OkvGgbp2XK_njdFh5bMw -O regs.zip

import shutil
# move regs.zip to reg_images folder
shutil.move('regs.zip', f'{reg_dir}')

# extract zip contents to current folder and delete zip
import zipfile
%cd $reg_dir
with zipfile.ZipFile('regs.zip', 'r') as zip_ref:
    zip_ref.extractall()

# delete zip
!rm regs.zip

/workspace/sd-scripts
Downloading...
From (uriginal): https://drive.google.com/uc?id=1CIqOhLBfzrl5OkvGgbp2XK_njdFh5bMw
From (redirected): https://drive.google.com/uc?id=1CIqOhLBfzrl5OkvGgbp2XK_njdFh5bMw&confirm=t&uuid=eb8d09f1-647b-4615-bee0-bddc3b3c1ac5
To: /workspace/sd-scripts/regs.zip
100%|████████████████████████████████████████| 639M/639M [00:08<00:00, 72.4MB/s]
/workspace/sd-scripts/reg_images/1_person


In [7]:
# back to parent folder
%cd /workspace/sd-scripts

# set accelerate config with defaults
!accelerate config default --mixed_precision "bf16"


/workspace/sd-scripts
2023-08-04 12:07:36.513471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 12:07:37.063887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!accelerate launch --num_cpu_threads_per_process=2 "./sdxl_train.py" \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="training_images" \
  --reg_data_dir="reg_images" \
  --output_dir="output" \
  --output_name="myProjectName" \
  --save_model_as="safetensors" \
  --train_batch_size=2 \
  --max_train_steps=2000 \
  --save_every_n_steps=500 \
  --optimizer_type="adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --xformers \
  --cache_latents \
  --lr_scheduler="constant_with_warmup" \
  --lr_warmup_steps=100 \
  --learning_rate=1e-5 \
  --max_grad_norm=0.0 \
  --train_text_encoder \
  --resolution="1024,1024" \
  --save_precision="float" \
  --save_n_epoch_ratio=1 \
  --max_data_loader_n_workers=1 \
  --persistent_data_loader_workers \
  --mixed_precision="bf16" \
  --full_bf16 \
  --logging_dir="logs" \
  --log_prefix="last" \
  --gradient_checkpointing
 